Hello Hello, just making a commit
In WSL! Again!

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam




from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

print(tf.__version__)


2.15.0


In [24]:
data = pd.read_csv('admissions_data.csv')
data = data.drop('Serial No.', axis=1)
print(data.head())
print(data.dtypes)
features = data.iloc[:,:-1]
labels = data.iloc[:,-1]
print(features.head())
print(labels.head())


   GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research  \
0        337          118                  4  4.5   4.5  9.65         1   
1        324          107                  4  4.0   4.5  8.87         1   
2        316          104                  3  3.0   3.5  8.00         1   
3        322          110                  3  3.5   2.5  8.67         1   
4        314          103                  2  2.0   3.0  8.21         0   

   Chance of Admit   
0              0.92  
1              0.76  
2              0.72  
3              0.80  
4              0.65  
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object
   GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research
0        337          118                  4  4.5   4.5  9.65         1
1        324          107  

In [25]:
features = data.iloc[:,:-1]
features_numerical = features.drop('Research', axis=1)
features_columns = features_numerical.columns
print(features.head())
print(features_columns)

   GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research
0        337          118                  4  4.5   4.5  9.65         1
1        324          107                  4  4.0   4.5  8.87         1
2        316          104                  3  3.0   3.5  8.00         1
3        322          110                  3  3.5   2.5  8.67         1
4        314          103                  2  2.0   3.0  8.21         0
Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA'], dtype='object')


In [26]:
labels = data.iloc[:,-1]
print(labels.head())

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64


In [27]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, random_state=4)

In [28]:
ct = ColumnTransformer([('numeric', StandardScaler(), features_columns)], remainder='passthrough')
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)
print(features_train_scaled[:5])
print(features_test_scaled[:5])

[[ 0.35791216  0.84271606 -0.94792265  0.17769404  0.05044464  0.39633165
   1.        ]
 [ 1.61060472  2.01253607  1.72479159  1.18731924  1.6718794   1.65487784
   1.        ]
 [-0.4473902  -0.4942211  -0.94792265 -0.83193117  0.05044464 -0.1004629
   0.        ]
 [ 0.35791216  0.50848177  1.72479159  0.68250664  0.59092289  1.20776274
   1.        ]
 [ 0.35791216  0.50848177  1.72479159  1.69213184  1.13140114  1.12496365
   1.        ]]
[[ 0.98425844  1.17695035 -0.0570179  -0.32711857 -0.49003362  0.79376729
   0.        ]
 [ 0.4473902   0.84271606  1.72479159  1.69213184  1.6718794   0.86000656
   1.        ]
 [-0.71582432  0.17424748 -0.0570179   0.17769404  0.05044464 -0.53101817
   0.        ]
 [ 1.3421706   1.51118464  1.72479159  0.68250664  0.59092289  1.19120293
   1.        ]
 [ 1.43164864  1.84541893  1.72479159  1.69213184  1.6718794   1.82047602
   1.        ]]


Ok let's build a deep net

In [29]:
admission_predictor = Sequential()
admission_predictor.add(InputLayer(input_shape = (features.shape[1])))
admission_predictor.add(Dense(units=64, activation='relu'))
admission_predictor.add(Dropout(0.2))
admission_predictor.add(Dense(units=32, activation='relu'))
admission_predictor.add(Dropout(0.2))
admission_predictor.add(Dense(units=1))
print(admission_predictor.summary)


<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7f2af05301d0>>


In [31]:
adam_optimizer = Adam(learning_rate = 0.01)
admission_predictor.compile(loss='mse', metrics='mae', optimizer = adam_optimizer)
admission_predictor.fit(features_train_scaled, labels_train, epochs=40, batch_size = 5, verbose = 1)
res_mse, res_mae = admission_predictor.evaluate(features_test_scaled, labels_test, verbose=0)
print(res_mse, res_mae)

Epoch 1/40


2024-01-17 16:02:52.778191: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2024-01-17 16:02:53.635247: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2a8d588600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-17 16:02:53.635286: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-01-17 16:02:53.648237: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-17 16:02:53.675130: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1705536173.743963   14781 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


75/75 [==============================] - 3s 5ms/step - loss: 0.0911 - mae: 0.2298
Epoch 2/40
75/75 [==============================] - 0s 4ms/step - loss: 0.0182 - mae: 0.1092
Epoch 3/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0141 - mae: 0.0937
Epoch 4/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0082 - mae: 0.0728
Epoch 5/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0067 - mae: 0.0651
Epoch 6/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0060 - mae: 0.0589
Epoch 7/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0054 - mae: 0.0561
Epoch 8/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0053 - mae: 0.0567
Epoch 9/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0052 - mae: 0.0572
Epoch 10/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0049 - mae: 0.0525
Epoch 11/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0056 